In [105]:
#STREAMS2TABLE
#written by Zachary J.J. Roper

#A script for converting raw event codes and timestamps into a trial-item dataset

In [106]:
#This notebook requires the following modules
import os
import numpy
import csv

In [107]:
#READ IN EVENT CODES
#The prepared file, out.csv, is a 2 column x session length (based on total number of trials) consistinting of event 
#codes (EVENTS[]) and their paired timestamps(TIMES[]). The file streams2text.m was used to generate these data 
#from the TDT data dump.

In [108]:
with open('out.csv') as csvfile:
    data = csv.reader(csvfile, delimiter = ',')
    events = []
    times = []
    for row in data:
        event = row[0]
        events.append(event)
        time = row[1]
        times.append(time)
pairedlist = list(zip(events,times))

In [109]:
events.count('2756') #catch trials

8

In [110]:
trial = 0
triallist = [None]*len(pairedlist)
eclist = [None]*len(pairedlist)
for i, data in enumerate(pairedlist):
    if pairedlist[i][0] == '2301':
        trial += 1
    triallist[i] = trial  
    eclist[i] = i
uberlist = list(zip(triallist,eclist,pairedlist))

In [111]:
#uberlist

In [112]:
'2810' in uberlist[74][2]

False

In [113]:
RT = [None]*len(pairedlist)
RTbeg = 0*len(pairedlist)
RTend = 0*len(pairedlist)
for i, trial in enumerate(uberlist):
    #print(i,trial)
    if '2651' in uberlist[i][2]:
        RTbeg = float(uberlist[i][2][1])
        focusTrial = uberlist[i][0]
        for p, case in enumerate(uberlist):
            if '2810' in uberlist[p][2] and uberlist[i][2] == focusTrial:
                RTend = float(uberlist[p][2][1])
    RT[i] = RTend - RTbeg

In [114]:
#RT

In [115]:
#The following provides indexed lists for each of the individual event codes of interest 

In [116]:
TargetOnset = []
FixOnset = []
Saccade = []
for i, x in enumerate(events):
    #print(i,x)
    if x == '2301':
        FixOnset.append(i)
    if x == '2651':
        TargetOnset.append(i)
    if x == '2810':
        Saccade.append(i)

In [117]:
#float(times[74])-float(times[70])

In [118]:
#Saccade

In [119]:
#for i, x in enumerate(Saccade):
#    print(Saccade[i],TargetOnset[i])

In [120]:
TOtimes = []
for items in TargetOnset:
    TOtimes.append(times[items])

In [121]:
Trials = []
TimeTrials = []
for i, x in enumerate(FixOnset):
    if i < len(FixOnset)-1:
        a = FixOnset[i]
        b = FixOnset[i+1]-1
        Trials.append(events[a:b])
        TimeTrials.append(times[a:b])
    else:
        a = FixOnset[i]
        Trials.append(events[a:])
        TimeTrials.append(times[a:])

In [122]:
#print(TimeTrials[0])

In [123]:
path = os.getcwd()
csvfile = (path+"/table.csv")
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(Trials)

In [124]:
accuracy = [None]*len(Trials)
abort = [None]*len(Trials)
error = [None]*len(Trials)
catch = [None]*len(Trials)
correct = [None]*len(Trials)
for i, trial in enumerate(Trials):
    if '2600' in trial:
        accuracy[i] = 1 #Correct Saccade (2600)
    else:
        accuracy[i] = 0  
    if '2651' in trial: #fixation was maintained sufficiently long enough for target onset (2651)
        abort[i] = 0
    else:
        abort[i] = 1
    if '887' in trial:
        error[i] = 1
    else:
        error[i] = 0
    if '2756' in trial:
        catch[i] = 1
    else:
        catch[i] = 0
    if '888' in trial:
        correct[i] = 1
    else:
        correct[i] = 0

In [125]:
new_dict = {}
for i, trial in enumerate(Trials):
    new_dict[i+1] = dict(zip(Trials[i],TimeTrials[i]))

In [126]:
RT = [0]*len(Trials)
focusTrial = 0
for i, trial in enumerate(Trials):
    if '2651' in trial and '2810' in trial: #and '2600' in trial:
        focusTrial = i + 1
        for p, case in enumerate(uberlist):
            if uberlist[p][0] == focusTrial:
                if '2810' in uberlist[p][2][0]:
                    #print(uberlist[p][2][1])
                    RTend = float(uberlist[p][2][1])
                if '2651' in uberlist[p][2][0]:
                    #print(uberlist[p][2][1])
                    RTbeg = float(uberlist[p][2][1])
            RT[i] = RTend - RTbeg

In [127]:
#RT

In [128]:
checksum = sum(error)+sum(abort)+sum(catch)

In [129]:
checksum

199

In [130]:
path = os.getcwd()
csvfile = (path+"/behaviorArray.csv")
trial = 0
triallist = [None]*len(abort)
with open(csvfile, "w") as output:
    for i,x in enumerate(abort):
        trial += 1
        triallist[i] = trial 
        print(i+1,abort[i],accuracy[i],error[i],correct[i],catch[i],RT[i])
    header = ['Trial', 'Abort', 'Accuracy', 'Error', 'Correct', 'Catch', 'RT']    
    dataset = list(zip(triallist,abort,accuracy,error,correct,catch,RT))
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(dataset)

1 0 0 0 1 0 41.98389999999927
2 1 0 0 0 0 0
3 1 0 0 0 0 0
4 1 0 0 0 0 0
5 0 1 0 1 0 205.98790000000008
6 1 0 0 0 0 0
7 1 0 0 0 0 0
8 1 0 0 0 0 0
9 1 0 0 0 0 0
10 1 0 0 0 0 0
11 1 0 0 0 0 0
12 1 0 0 0 0 0
13 1 0 0 0 0 0
14 0 1 0 1 0 217.98919999999998
15 1 0 0 0 0 0
16 0 0 1 0 0 198.0007000000005
17 1 0 0 0 0 0
18 0 1 0 1 0 191.9794000000038
19 0 1 0 1 0 214.0570000000007
20 0 1 0 1 0 213.97490000000107
21 0 1 0 1 0 166.01110000000335
22 0 1 0 1 0 189.97260000000097
23 0 1 0 1 0 199.96659999999974
24 0 1 0 1 0 191.97959999999875
25 1 0 0 0 0 0
26 1 0 0 0 0 0
27 0 1 0 1 0 169.9841000000015
28 0 1 0 1 0 84.00890000000072
29 0 1 0 1 0 161.9968000000008
30 1 0 0 0 0 0
31 1 0 0 0 0 0
32 1 0 0 0 0 0
33 0 1 0 1 0 173.99790000000212
34 0 1 0 1 0 203.98099999999977
35 0 1 0 1 0 154.00950000000012
36 0 1 0 1 0 189.97240000000602
37 0 1 0 1 0 171.99090000000433
38 0 1 0 1 0 169.9841000000015
39 1 0 0 0 0 0
40 0 1 0 1 0 218.03029999999853
41 0 1 0 1 0 167.976800000004
42 1 0 0 0 0 0
43 0 0 0 1 0 21

In [131]:
sum(catch)

8